## 7장　벽돌깨기 게임 학습 결과 실행용 프로그램  

In [2]:
# 구현에 사용할 패키지 임포트
import numpy as np
from collections import deque
from tqdm import tqdm

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import gym
from gym import spaces
from gym.spaces.box import Box


In [2]:
# 구현에 사용할 패키지 임포트
import matplotlib.pyplot as plt
%matplotlib inline


# 애니메이션을 만드는 함수
# 참고 URL http://nbviewer.jupyter.org/github/patrickmineault
# /xcorr-notebooks/blob/master/Render%20OpenAI%20gym%20as%20GIF.ipynb
from JSAnimation.IPython_display import display_animation
from matplotlib import animation
from IPython.display import display

def display_frames_as_gif(frames):
    """
    Displays a list of frames as a gif, with controls
    """
    plt.figure(figsize=(frames[0].shape[1]/72.0*1, frames[0].shape[0]/72.0*1),
               dpi=72)
    patch = plt.imshow(frames[0])
    plt.axis('off')
 
    def animate(i):
        patch.set_data(frames[i])
 
    anim = animation.FuncAnimation(plt.gcf(), animate, frames=len(frames),
                                   interval=20)
 
    anim.save('breakout.mp4')  # 애니메이션을 저장하는 부분
    display(display_animation(anim, default_mode='loop'))
    

In [3]:
# 실행환경 설정
# 参考：https://github.com/openai/baselines/blob/master/baselines/common/atari_wrappers.py

import cv2
cv2.ocl.setUseOpenCL(False)


class NoopResetEnv(gym.Wrapper):
    def __init__(self, env, noop_max=30):
        '''첫 번째 트릭 No-Operation. 초기화 후 일정 단계에 이를때까지 아무 행동도 하지않고
        게임 초기 상태를 다양하게 하여 특정 시작 상태만 학습하는 것을 방지한다'''

        gym.Wrapper.__init__(self, env)
        self.noop_max = noop_max
        self.override_num_noops = None
        self.noop_action = 0
        assert env.unwrapped.get_action_meanings()[0] == 'NOOP'

    def reset(self, **kwargs):
        """ Do no-op action for a number of steps in [1, noop_max]."""
        self.env.reset(**kwargs)
        if self.override_num_noops is not None:
            noops = self.override_num_noops
        else:
            noops = self.unwrapped.np_random.randint(
                1, self.noop_max + 1)  # pylint: disable=E1101
        assert noops > 0
        obs = None
        for _ in range(noops):
            obs, _, done, _ = self.env.step(self.noop_action)
            if done:
                obs = self.env.reset(**kwargs)
        return obs

    def step(self, ac):
        return self.env.step(ac)


class EpisodicLifeEnv(gym.Wrapper):
    def __init__(self, env):
        '''두 번째 트릭 Episodic Life. 한번 실패를 게임 종료로 간주하나, 다음 게임을 같은 블록 상태로 시작'''
        gym.Wrapper.__init__(self, env)
        self.lives = 0
        self.was_real_done = True

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        self.was_real_done = done
        # check current lives, make loss of life terminal,
        # then update lives to handle bonus lives
        lives = self.env.unwrapped.ale.lives()
        if lives < self.lives and lives > 0:
            # for Qbert sometimes we stay in lives == 0 condtion for a few frames
            # so its important to keep lives > 0, so that we only reset once
            # the environment advertises done.
            done = True
        self.lives = lives
        return obs, reward, done, info

    def reset(self, **kwargs):
        '''5번 실패하면 게임을 완전히 다시 시작'''
        if self.was_real_done:
            obs = self.env.reset(**kwargs)
        else:
            # no-op step to advance from terminal/lost life state
            obs, _, _, _ = self.env.step(0)
        self.lives = self.env.unwrapped.ale.lives()
        return obs


class MaxAndSkipEnv(gym.Wrapper):
    def __init__(self, env, skip=4):
        '''세 번째 트릭 Max and Skip. 4프레임 동안 같은 행동을 지속하되, 3번째와 4번째 프레임의 최댓값 이미지를 관측 obs로 삼는다'''
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros(
            (2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break
        # Note that the observation on the done=True frame
        # doesn't matter
        max_frame = self._obs_buffer.max(axis=0)

        return max_frame, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


class WarpFrame(gym.ObservationWrapper):
    def __init__(self, env):
        '''네 번째 트릭 Warp frame. DQN 네이처 논문 구현과 같이 84*84 흑백 이미지를 사용'''
        gym.ObservationWrapper.__init__(self, env)
        self.width = 84
        self.height = 84
        self.observation_space = spaces.Box(low=0, high=255,
                                            shape=(self.height, self.width, 1), dtype=np.uint8)

    def observation(self, frame):
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2GRAY)
        frame = cv2.resize(frame, (self.width, self.height),
                           interpolation=cv2.INTER_AREA)
        return frame[:, :, None]


class WrapPyTorch(gym.ObservationWrapper):
    def __init__(self, env=None):
        '''인덱스 순서를 파이토치 미니배치와 같이 조정하는 래퍼'''
        super(WrapPyTorch, self).__init__(env)
        obs_shape = self.observation_space.shape
        self.observation_space = Box(
            self.observation_space.low[0, 0, 0],
            self.observation_space.high[0, 0, 0],
            [obs_shape[2], obs_shape[1], obs_shape[0]],
            dtype=self.observation_space.dtype)

    def observation(self, observation):
        return observation.transpose(2, 0, 1)


In [4]:
# 再生用の実行環境


class EpisodicLifeEnvPlay(gym.Wrapper):
    def __init__(self, env):
        '''두 번째 트릭 Episodic Life. 한번 실패를 게임 종료로 간주하나, 다음 게임을 같은 블록 상태로 시작
        그러나 여기서는 학습 결과 플레이를 위해 한번 실패에도 블록 상태까지 초기화한다'''

        gym.Wrapper.__init__(self, env)

    def step(self, action):
        obs, reward, done, info = self.env.step(action)
        # 처음 5개 라이프를 갖고 시작하지만, 하나만 잃어도 종료한다
        if self.env.unwrapped.ale.lives() < 5:
            done = True

        return obs, reward, done, info

    def reset(self, **kwargs):
        '''한번이라도 실패하면 완전히 초기화'''

        obs = self.env.reset(**kwargs)

        return obs


class MaxAndSkipEnvPlay(gym.Wrapper):
    def __init__(self, env, skip=4):
        '''세 번째 트릭 Max and Skip. 4프레임 동안 같은 행동을 지속하되, 4번째 프레임 이미지를 관측 obs로 삼는다'''
        gym.Wrapper.__init__(self, env)
        # most recent raw observations (for max pooling across time steps)
        self._obs_buffer = np.zeros(
            (2,)+env.observation_space.shape, dtype=np.uint8)
        self._skip = skip

    def step(self, action):
        """Repeat action, sum reward, and max over last observations."""
        total_reward = 0.0
        done = None
        for i in range(self._skip):
            obs, reward, done, info = self.env.step(action)
            if i == self._skip - 2:
                self._obs_buffer[0] = obs
            if i == self._skip - 1:
                self._obs_buffer[1] = obs
            total_reward += reward
            if done:
                break

        return obs, total_reward, done, info

    def reset(self, **kwargs):
        return self.env.reset(**kwargs)


In [5]:
# 실행환경 생성 함수

# 병렬 실행환경
from baselines.common.vec_env.subproc_vec_env import SubprocVecEnv


def make_env(env_id, seed, rank):
    def _thunk():
        '''멀티 프로세스로 동작하는 환경 SubprocVecEnv를 실행하기 위해 필요하다'''

        env = gym.make(env_id)
        #env = NoopResetEnv(env, noop_max=30)
        env = MaxAndSkipEnv(env, skip=4)
        env.seed(seed + rank)  # 난수 시드 설정
        #env = EpisodicLifeEnv(env)
        env = EpisodicLifeEnvPlay(env)
        env = WarpFrame(env)
        env = WrapPyTorch(env)

        return env

    return _thunk


def make_env_play(env_id, seed, rank):
    '''학습 결과 시연용 실행환경'''
    env = gym.make(env_id)
    #env = NoopResetEnv(env, noop_max=30)
    #env = MaxAndSkipEnv(env, skip=4)
    env = MaxAndSkipEnvPlay(env, skip=4)
    env.seed(seed + rank)  # 난수 시드 설정
    env = EpisodicLifeEnvPlay(env)
    #env = WarpFrame(env)
    #env = WrapPyTorch(env)

    return env


In [6]:
# 상수 정의

ENV_NAME = 'BreakoutNoFrameskip-v4' 
# Breakout-v0 대신 BreakoutNoFrameskip-v4을 사용
# v0은 2~4개 프레임을 자동으로 생략하므로 이 기능이 없는 버전을 사용한다
# 참고 URL https://becominghuman.ai/lets-build-an-atari-ai-part-1-dqn-df57e8ff3b26
# https://github.com/openai/gym/blob/5cb12296274020db9bb6378ce54276b31e7002da/gym/envs/__init__.py#L371
    
NUM_SKIP_FRAME = 4 # 생략할 프레임 수
NUM_STACK_FRAME = 4  # 하나의 상태로 사용할 프레임의 수
NOOP_MAX = 30  #  초기화 후 No-operation을 적용할 최초 프레임 수의 최댓값
NUM_PROCESSES = 16 #  병렬로 실행할 프로세스 수
NUM_ADVANCED_STEP = 5  # Advanced 학습할 단계 수
GAMMA = 0.99  # 시간할인율

TOTAL_FRAMES=10e6  #  학습에 사용하는 총 프레임 수
NUM_UPDATES = int(TOTAL_FRAMES / NUM_ADVANCED_STEP / NUM_PROCESSES)  # 신경망 수정 총 횟수
# NUM_UPDATES는 약 125,000이 됨


In [7]:
# A2C 손실함수를 계산하기 위한 상수
value_loss_coef = 0.5
entropy_coef = 0.01
max_grad_norm = 0.5

# 최적회 기법 RMSprop에 대한 설정
lr = 7e-4
eps = 1e-5
alpha = 0.99


In [3]:
# GPU 사용 설정
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)


cuda


In [9]:
# 메모리 클래스 정의


class RolloutStorage(object):
    '''Advantage 학습에 사용하는 메모리 클래스'''

    def __init__(self, num_steps, num_processes, obs_shape):

        self.observations = torch.zeros(
            num_steps + 1, num_processes, *obs_shape).to(device)
        # *로 리스트의 요소를 풀어낸다(unpack)
        # obs_shape→(4,84,84)
        # *obs_shape→ 4 84 84

        self.masks = torch.ones(num_steps + 1, num_processes, 1).to(device)
        self.rewards = torch.zeros(num_steps, num_processes, 1).to(device)
        self.actions = torch.zeros(
            num_steps, num_processes, 1).long().to(device)

        # 할인 총보상을 저장
        self.returns = torch.zeros(num_steps + 1, num_processes, 1).to(device)
        self.index = 0  # 저장할 인덱스

    def insert(self, current_obs, action, reward, mask):
        '''인덱스가 가리키는 다음 자리에 transition을 저장'''
        self.observations[self.index + 1].copy_(current_obs)
        self.masks[self.index + 1].copy_(mask)
        self.rewards[self.index].copy_(reward)
        self.actions[self.index].copy_(action)

        self.index = (self.index + 1) % NUM_ADVANCED_STEP  # 인덱스 업데이트

    def after_update(self):
        '''Advantage 학습 단계 수만큼 단계가 진행되면 가장 최근 단계를 index0에 저장'''
        self.observations[0].copy_(self.observations[-1])
        self.masks[0].copy_(self.masks[-1])

    def compute_returns(self, next_value):
        '''Advantage 학습 단계에 들어가는 각 단계에 대해 할인 총보상을 계산'''

        # 주의 : 5번째 단계부터 거슬러 올라가며 계산
        # 주의 : 5번째 단계가 Advantage1, 4번째 단계가 Advantage2가 되는 식임
        self.returns[-1] = next_value
        for ad_step in reversed(range(self.rewards.size(0))):
            self.returns[ad_step] = self.returns[ad_step + 1] * \
                GAMMA * self.masks[ad_step + 1] + self.rewards[ad_step]


In [10]:
# A2C 신경망 구성


def init(module, gain):
    '''결합 가중치를 초기화하는 함수'''
    nn.init.orthogonal_(module.weight.data, gain=gain)
    nn.init.constant_(module.bias.data, 0)
    return module


class Flatten(nn.Module):
    '''합성곱층의 출력 이미지를 1차원으로 변환하는 층'''

    def forward(self, x):
        return x.view(x.size(0), -1)


class Net(nn.Module):
    def __init__(self, n_out):
        super(Net, self).__init__()

        # 결합 가중치 초기화 함수
        def init_(module): return init(
            module, gain=nn.init.calculate_gain('relu'))

        # 합성곱층을 정의
        self.conv = nn.Sequential(
            # 이미지 크기의 변화 (84*84 -> 20*20)
            init_(nn.Conv2d(NUM_STACK_FRAME, 32, kernel_size=8, stride=4)),
            # 프레임 4개를 합치므로 input=NUM_STACK_FRAME=4가 된다. 출력은 32이다.
            # size 계산  size = (Input_size - Kernel_size + 2*Padding_size)/ Stride_size + 1

            nn.ReLU(),
            # 이미지 크기의 변화 (20*20 -> 9*9)
            init_(nn.Conv2d(32, 64, kernel_size=4, stride=2)),
            nn.ReLU(),
            init_(nn.Conv2d(64, 64, kernel_size=3, stride=1)),  # 이미지 크기의 변화(9*9 -> 7*7)
            nn.ReLU(),
            Flatten(),  # 이미지를 1차원으로 변환
            init_(nn.Linear(64 * 7 * 7, 512)),  # 7*7 이미지 64개를 512차원으로 변환
            nn.ReLU()
        )

        # 결합 가중치 초기화 함수
        def init_(module): return init(module, gain=1.0)

        # Critic을 정의
        self.critic = init_(nn.Linear(512, 1))  # 출력은 상태가치이므로 1개

        # 결합 가중치 초기화 함수
        def init_(module): return init(module, gain=0.01)

        # Actor를 정의
        self.actor = init_(nn.Linear(512, n_out))  # 출력이 행동이므로 출력 수는 행동의 가짓수
        
        # 신경망을 학습 모드로 전환
        self.train()

    def forward(self, x):
        '''신경망의 순전파 계산 정의'''
        input = x / 255.0  # 이미지의 픽셀값을 [0,255]에서 [0,1] 구간으로 정규화
        conv_output = self.conv(input)  # 합성곱층 계산
        critic_output = self.critic(conv_output)  # 상태가치 출력 계산
        actor_output = self.actor(conv_output)  # 행동 출력 계산

        return critic_output, actor_output

    def act(self, x):
        '''상태 x일때 취할 확률을 확률적으로 구함'''
        value, actor_output = self(x)
        probs = F.softmax(actor_output, dim=1)    # dim=1で行動の種類方向に計算
        action = probs.multinomial(num_samples=1)

        return action

    def get_value(self, x):
        '''상태 x의 상태가치를 구함'''
        value, actor_output = self(x)

        return value

    def evaluate_actions(self, x, actions):
        '''상태 x의 상태가치, 실제 행동 actions의 로그 확률, 엔트로피를 구함'''
        value, actor_output = self(x)

        log_probs = F.log_softmax(actor_output, dim=1)  # dim=1이므로 행동의 종류 방향으로 계산
        action_log_probs = log_probs.gather(1, actions)  # 실제 행동에 대한 log_probs 계산

        probs = F.softmax(actor_output, dim=1)  # dim=1이므로 행동의 종류 방향으로 계산
        dist_entropy = -(log_probs * probs).sum(-1).mean()

        return value, action_log_probs, dist_entropy


In [11]:
# 에이전트의 두뇌 역할을 하는 클래스로, 모든 에이전트가 공유한다


class Brain(object):
    def __init__(self, actor_critic):

        self.actor_critic = actor_critic  # actor_critic은 Net클래스로 구현한 신경망이다

        # 이미 학습된 결합 가중치를 로드하려면
        filename = 'weight_end.pth'
        #filename = 'weight_112500.pth'
        param = torch.load(filename, map_location='cpu')
        self.actor_critic.load_state_dict(param)

        # 가중치를 학습하는 최적화 알고리즘 설정
        self.optimizer = optim.RMSprop(
            actor_critic.parameters(), lr=lr, eps=eps, alpha=alpha)

    def update(self, rollouts):
        '''advanced 학습 대상 5단계를 모두 사용하여 수정한다'''
        obs_shape = rollouts.observations.size()[2:]  # torch.Size([4, 84, 84])
        num_steps = NUM_ADVANCED_STEP
        num_processes = NUM_PROCESSES

        values, action_log_probs, dist_entropy = self.actor_critic.evaluate_actions(
            rollouts.observations[:-1].view(-1, *obs_shape),
            rollouts.actions.view(-1, 1))

        # 각 변수의 크기에 주의할 것
        # rollouts.observations[:-1].view(-1, *obs_shape) torch.Size([80, 4, 84, 84])
        # rollouts.actions.view(-1, 1) torch.Size([80, 1])
        # values torch.Size([80, 1])
        # action_log_probs torch.Size([80, 1])
        # dist_entropy torch.Size([])

        values = values.view(num_steps, num_processes,
                             1)  # torch.Size([5, 16, 1])
        action_log_probs = action_log_probs.view(num_steps, num_processes, 1)

        advantages = rollouts.returns[:-1] - values  # torch.Size([5, 16, 1])
        value_loss = advantages.pow(2).mean()

        action_gain = (advantages.detach() * action_log_probs).mean()
        # advantages는 detach 하여 정수로 취급한다

        total_loss = (value_loss * value_loss_coef -
                      action_gain - dist_entropy * entropy_coef)

        self.optimizer.zero_grad()  # 경사 초기화
        total_loss.backward()  # 역전파 계산
        nn.utils.clip_grad_norm_(self.actor_critic.parameters(), max_grad_norm)
        # 한번에 결합 가중치가 너무 크게 변화하지 않도록, 경사의 최댓값을 0.5로 제한한다

        self.optimizer.step()  # 결합 가중치 수정


In [12]:
# Breakout을 실행하는 환경 클래스

NUM_PROCESSES = 1


class Environment:
    def run(self):

        # 난수 시드 설정
        seed_num = 1
        torch.manual_seed(seed_num)
        if use_cuda:
            torch.cuda.manual_seed(seed_num)

        # 실행환경 구축
        torch.set_num_threads(seed_num)
        envs = [make_env(ENV_NAME, seed_num, i) for i in range(NUM_PROCESSES)]
        envs = SubprocVecEnv(envs)  # 멀티프로세스 실행환경

        # 모든 에이전트가 공유하는 두뇌 역할 클래스 Brain 객체 생성
        n_out = envs.action_space.n  # 행동의 가짓수는 4
        actor_critic = Net(n_out).to(device)  # GPU 사용
        global_brain = Brain(actor_critic)

        # 정보 저장용 변수 생성
        obs_shape = envs.observation_space.shape  # (1, 84, 84)
        obs_shape = (obs_shape[0] * NUM_STACK_FRAME,
                     *obs_shape[1:])  # (4, 84, 84)
        # torch.Size([16, 4, 84, 84])
        current_obs = torch.zeros(NUM_PROCESSES, *obs_shape).to(device)
        rollouts = RolloutStorage(
            NUM_ADVANCED_STEP, NUM_PROCESSES, obs_shape)  # rollouts 객체
        episode_rewards = torch.zeros([NUM_PROCESSES, 1])  # 현재 에피소드에서 받을 보상 저장
        final_rewards = torch.zeros([NUM_PROCESSES, 1])  # 마지막 에피소드의 총 보상 저장

        # 초기 상태로 시작
        obs = envs.reset()
        obs = torch.from_numpy(obs).float()  # torch.Size([16, 1, 84, 84])
        current_obs[:, -1:] = obs  # 4번째 프레임에 가장 최근 관측결과를 저장

        # advanced 학습에 사용할 객체 rollouts에 첫번째 상태로 현재 상태를 저장
        rollouts.observations[0].copy_(current_obs)

        # 애니메이션 생성용 환경(시연용 추가)
        env_play = make_env_play(ENV_NAME, seed_num, 0)
        obs_play = env_play.reset()

        # 애니메이션 생성을 위해 이미지를 저장할 변수(시연용 추가)
        frames = []
        main_end = False

        # 주 반복문
        for j in tqdm(range(NUM_UPDATES)):

            # 보상이 기준을 넘어서면 종료 (시연용 추가)
            if main_end:
                break

            # advanced 학습 범위에 들어가는 단계마다 반복
            for step in range(NUM_ADVANCED_STEP):

                # 행동을 결정
                with torch.no_grad():
                    action = actor_critic.act(rollouts.observations[step])

                cpu_actions = action.squeeze(1).cpu().numpy()  # tensor를 NumPy 변수로

                # 1단계를 병렬로 실행, 반환값 obs의 크기는 (16, 1, 84, 84)
                obs, reward, done, info = envs.step(cpu_actions)

                # 보상을 텐서로 변환한 다음 에피소드 총 보상에 더함
                # 크기가 (16,)인 것을 (16, 1)로 변환
                reward = np.expand_dims(np.stack(reward), 1)
                reward = torch.from_numpy(reward).float()
                episode_rewards += reward

                # 각 프로세스마다 done이 True이면 0, False이면 1
                masks = torch.FloatTensor(
                    [[0.0] if done_ else [1.0] for done_ in done])

                # 마지막 에피소드의 총 보상을 업데이트
                final_rewards *= masks  # done이 True이면 0을 곱하고, False이면 1을 곱하여 리셋
                # done이 False이면 0을 더하고, True이면 epicodic_rewards를 더함
                final_rewards += (1 - masks) * episode_rewards

                # 이미지를 구함(시연용 추가）
                obs_play, reward_play, _, _ = env_play.step(cpu_actions[0])
                frames.append(obs_play)  # 변환한 이미지를 저장
                if done[0]:  # 첫번째 프로세스가 종료된 경우
                    print(episode_rewards[0][0].numpy())  # 보상

                    # 보상이 300을 초과하면 종료
                    if (episode_rewards[0][0].numpy()) > 300:
                        main_end = True
                        break
                    else:
                        obs_view = env_play.reset()
                        frames = []  # 저장한 이미지를 리셋

                # 에피소드의 총 보상을 업데이트
                episode_rewards *= masks  # 각 프로세스마다 done이 True이면 0, False이면 1을 곱함

                # masks 변수를 GPU로 전달
                masks = masks.to(device)

                # done이 True이면 모두 0으로
                # mask의 크기를 torch.Size([16, 1]) --> torch.Size([16, 1, 1 ,1])로 변환하고 곱함
                current_obs *= masks.unsqueeze(2).unsqueeze(2)

                # 프레임을 모음
                # torch.Size([16, 1, 84, 84])
                obs = torch.from_numpy(obs).float()
                current_obs[:, :-1] = current_obs[:, 1:]  # 0～2번째 프레임을 1~3번째 프레임으로 덮어씀
                current_obs[:, -1:] = obs  # 4번째 프레임에 가장 최근 obs를 저장

                # 메모리 객체에 현 단계의 transition을 저장
                rollouts.insert(current_obs, action.data, reward, masks)

            # advanced 학습의 for문 끝

            # advanced 학습 대상 단계 중 마지막 단계의 상태에서 예상되는 상태가치를 계산
            with torch.no_grad():
                next_value = actor_critic.get_value(
                    rollouts.observations[-1]).detach()

            # 모든 단계의 할인 총보상을 계산하고, rollouts의 변수 returns를 업데이트
            rollouts.compute_returns(next_value)

            # 신경망 수정 및 rollout 업데이트
            # global_brain.update(rollouts)
            rollouts.after_update()

        # 주 반복문 끝
        display_frames_as_gif(frames)  # 애니메이션 저장 및 재생


In [ ]:
# 실행
breakout_env = Environment()
frames = breakout_env.run()